In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

import glob
import numpy as np
import scipy.optimize
import pandas as pd

import skimage.io
import skimage.filters

import bokeh.plotting
import holoviews as hv
hv.extension('bokeh')
import panel as pn

import bebi103.image
import lac

## Testing filtering of bleaching time series movies
Two questions to address here.
 - Both iXon cameras have a slow timescale drift, affecting the entire image. Can we remove this by simply subtracting off the median of each frame?
 - What does our SNR look like, before & after a light-to-moderate Gaussian blur?
 
First let's load a single time-series of images as a stack.

In [ ]:
root_data_path = "/Users/muir/datasets/sm_lac/"
im_path = "20200127/O1_0.4ngmL_TIRFobj_epi_1/Pos0/"

im_fname_list = sorted(glob.glob(root_data_path + im_path + "*mNeon*.tif"))
im_bfname_list = sorted(glob.glob(root_data_path + im_path + "*Brightfield*.tif"))
im_stack = skimage.io.imread_collection(im_fname_list)
im_bfstack = skimage.io.imread_collection(im_bfname_list)

Now with the images loaded, subtract off medians frame-by-frame. Then compute a Gaussian blur on this median subtracted image. Store them separately since I want to compare them visually.

In [ ]:
# preallocate storage for the median-subtracted & Gaussian-blurred stacks
im_stack_med_sub = np.empty_like(im_stack, dtype=float)
im_stack_gblur = np.empty_like(im_stack, dtype=float)
for i, im in enumerate(im_stack):
    im_stack_med_sub[i] = im_stack[i] - np.median(im_stack[i])
    im_stack_gblur[i] = skimage.filters.gaussian(im_stack_med_sub[i], sigma=0.7)

Next we'll show the median filtered vs. median filtered and Gaussian-blurred images side by side. A Panel slider widget selects what time point to show.

In [ ]:
time_slider = pn.widgets.IntSlider(name="time point", start=0, end=len(im_stack))
min_intens = 0
max_intens = 4000

# p = bokeh.plotting.figure()
@pn.depends(time_slider.param.value)
def show_ims(t):
#     im = im_stack_med_sub[t]
    z_scale = hv.Dimension("z", range=(min_intens, max_intens))
    layout = (hv.Image(im_stack_med_sub[t], vdims=z_scale)
              + hv.Image(im_stack_gblur[t], vdims=z_scale))
    return layout.opts(hv.opts.Image(height=600, width=600, tools=["hover"],
                                     colorbar=True, active_tools=['wheel_zoom']))

In [ ]:
pn.Column(time_slider, show_ims)

This actually looks fairly promising, even for the O1 construct in epi. I haven't even done a flatfield correction here. I think the SNR needs to be better, but we may not be way far off.

My biggest concern here is spots that appear to be disappearing and reappearing on the few second timescale. Is this real, or is this b/c the cell prep on Jan 27 was sloppy (cells only weakly stuck to glass, sometimes flapping the in the wind)? Or is this just the natural intensity fluctuations of single fluorophores? Or is this laser intensity fluctuations, either from the diode itself or from loose optics/dust in the beam path?

Or am I fooling myself and we're actually screwed? Are most of the spots I'm seeing actually a larger number of fluorophores and the single-molecule puncta are still hidden below noise? It's easy for the eye to be drawn to bright spots, but those highly expressing cells are exactly the ones we _don't_ want to consider here.

Really ought to repeat this with cells more firmly attached to the surface to try & distinguish these possible causes.

### Next step
After talking with HJ, I think the focus was way off in these movies and that this may be the cause of the fluctuating puncta brightness. Before we make decisions about switching to a new scope set up, we should repeat this test with better focus & see if the spot intensities are less variable. If they are less variable, we're probably best off staying with the Nikon & time-sharing. If they are still quite variable, we probably want to consider switching to a repurposed TPM Olympus so we can clean up the illumination beam w/ some combo of multimode fiber, diffuser, and lens array. The lack of PFS would suck, but it's not instantly a dealbreaker.

To verify this, just take a quick look at brightfield images. Am I as far out of focal plane as I suspect?

In [ ]:
time_slider = pn.widgets.IntSlider(name="time point", start=0, end=len(im_bfstack))
# min_intens = 0
# max_intens = 4000

# p = bokeh.plotting.figure()
@pn.depends(time_slider.param.value)
def show_ims(t):
    im = im_bfstack[t]
#     z_scale = hv.Dimension("z", range=(min_intens, max_intens))
#     layout = (hv.Image(im_stack_med_sub[t], vdims=z_scale)
#               + hv.Image(im_stack_gblur[t], vdims=z_scale))
#     return layout.opts(hv.opts.Image(height=600, width=600, tools=["hover"],
#                                      colorbar=True, active_tools=['wheel_zoom']))
    return hv.Image(im).opts(height=600, width=600, tools=["hover"], colorbar=True, active_tools=['wheel_zoom'])

In [ ]:
pn.Column(time_slider, show_ims)

Yup, according to HJ that is not at all what brightfield should look like.